In [1]:
!pip install -q unsloth peft transformers datasets accelerate rouge_score nltk scikit-learn evaluate polars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.7 MB/s eta 0:00

In [2]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

model_name = "unsloth/SmolLM-1.7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

In [3]:
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=512,
    dtype=torch.bfloat16,  # Use fp16 if bf16 not supported
    load_in_4bit=False,    # No quantization – full precision
)

==((====))==  Unsloth 2025.6.8: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
import polars as pl
from datasets import Dataset, concatenate_datasets

anime_list = [
    "chainsawman", "kurokonobasuke", "onepunch", "hellsing", "frieren", "aot",
    "naruto", "dr_stone", "gundam_00", "darling-in-the-franxx",
    "berserk", "evangelion", "onepiece"
]

train_parts, val_parts = [], []

for anime in anime_list:
    train_df = pl.read_ndjson(f"hf://datasets/theblackcat102/anime-understanding-dataset/{anime}_dev.jsonl").to_pandas()
    val_df = pl.read_ndjson(f"hf://datasets/theblackcat102/anime-understanding-dataset/{anime}_val.jsonl").to_pandas()
    train_parts.append(Dataset.from_pandas(train_df))
    val_parts.append(Dataset.from_pandas(val_df))

train_dataset = concatenate_datasets(train_parts)
val_dataset = concatenate_datasets(val_parts)

In [5]:
def format_prompt(example):
    question = example['question']
    options = f"A) {example['A']}\nB) {example['B']}\nC) {example['C']}\nD) {example['D']}"
    answer = example[example['answer']]

    prompt = f"Question: {question}\nOptions:\n{options}\nAnswer:"
    full_text = prompt + " " + answer

    tokens = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

train_dataset = train_dataset.map(format_prompt, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(format_prompt, remove_columns=val_dataset.column_names)

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

In [10]:
lora_config.r

16

In [13]:
from peft import LoraConfig
from unsloth import FastLanguageModel

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

model = FastLanguageModel.get_peft_model(
    model,
    r=lora_config.r,
    target_modules=lora_config.target_modules,
    lora_alpha=lora_config.lora_alpha,
    lora_dropout=lora_config.lora_dropout,
    bias=lora_config.bias,
)
model.print_trainable_parameters()

Unsloth 2025.6.8 patched 24 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


trainable params: 6,291,456 || all params: 1,717,667,840 || trainable%: 0.3663


In [15]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./unsloth_anime_qa_lora",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    save_strategy="epoch",
    eval_strategy="epoch",
    logging_steps=20,
    fp16=False,
    bf16=True,
    save_total_limit=2,
    report_to="none",
)

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collator,
)

In [17]:
trainer.train()
trainer.save_model("./unsloth_anime_qa_lora/final")
tokenizer.save_pretrained("./unsloth_anime_qa_lora/final")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 65 | Num Epochs = 3 | Total steps = 27
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 6,291,456/1,717,667,840 (0.37% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss
1,No log,2.727022
2,No log,2.667948
3,6.818600,2.649587


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


('./unsloth_anime_qa_lora/final/tokenizer_config.json',
 './unsloth_anime_qa_lora/final/special_tokens_map.json',
 './unsloth_anime_qa_lora/final/chat_template.jinja',
 './unsloth_anime_qa_lora/final/vocab.json',
 './unsloth_anime_qa_lora/final/merges.txt',
 './unsloth_anime_qa_lora/final/added_tokens.json',
 './unsloth_anime_qa_lora/final/tokenizer.json')

In [18]:
import evaluate
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import numpy as np
import math
import torch.nn.functional as F
import torch

In [19]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
smoothing = SmoothingFunction().method1
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [20]:
def compute_perplexity(model, tokenizer, input_text):
    tokens = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    input_ids = tokens["input_ids"].to(model.device)
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    return torch.exp(outputs.loss).item()

In [21]:
predictions, references, perplexities = [], [], []

sample_eval = val_dataset.select(range(15))
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(49152, 2048, padding_idx=2)
        (layers): ModuleList(
          (0-23): 24 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj)

In [22]:
for ex in sample_eval:
    input_ids = torch.tensor([ex["input_ids"]]).to(model.device)
    with torch.no_grad():
        generated = model.generate(
            input_ids=input_ids,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id
        )
    gen_text = tokenizer.decode(generated[0][len(input_ids[0]):], skip_special_tokens=True).strip()
    label_ids = [i for i in ex["labels"] if i != -100]
    ref_text = tokenizer.decode(label_ids, skip_special_tokens=True).strip()

    # Store
    predictions.append(gen_text)
    references.append([ref_text])
    perplexities.append(compute_perplexity(model, tokenizer, ref_text))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [23]:
# Final Scores
bleu_score = bleu.compute(predictions=predictions, references=references)
rouge_score = rouge.compute(predictions=predictions, references=[r[0] for r in references])
avg_ppl = np.mean(perplexities)

print(f"\nEvaluation Results:")
print(f"BLEU Score      : {bleu_score['bleu']:.4f}")
print(f"ROUGE-1         : {rouge_score['rouge1']:.4f}")
print(f"ROUGE-L         : {rouge_score['rougeL']:.4f}")
print(f"Perplexity      : {avg_ppl:.2f}")


Evaluation Results:
BLEU Score      : 0.1182
ROUGE-1         : 0.2786
ROUGE-L         : 0.2547
Perplexity      : 15.92


In [24]:
def infer_anime_question(model, tokenizer, question, options):
    prompt = f"Question: {question}\nOptions:\nA. {options['A']}\nB. {options['B']}\nC. {options['C']}\nD. {options['D']}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50, temperature=0.7, top_p=0.9)
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
    return response.strip(), compute_perplexity(model, tokenizer, response)

# Example
q = "Who is the main protagonist in One Punch Man?"
opt = {"A": "Saitama", "B": "Genos", "C": "Tornado", "D": "Garou"}

answer, ppl = infer_anime_question(model, tokenizer, q, opt)
print(f"Answer: {answer}\nPerplexity: {ppl:.2f}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Answer: A. Saitama
Perplexity: 551.57
